In [ ]:
import netgen.gui
from ngsolve import *
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import spsolve
from scipy import random

In [ ]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
Draw (mesh)


In [ ]:
fes = H1(mesh, order=3,dirichlet=".*")
gfu_jac = GridFunction(fes)
gfu_dir = GridFunction(fes)


In [ ]:
def SolveCG(mat, pre , sol, rhs, maxsteps, tol, printrate):
    u0=sol.CreateVector()
    u0.data=sol.data 
    d0=sol.CreateVector()
    d0.data=rhs-mat*u0.data
    p0=sol.CreateVector()
    p0.data=pre*d0.data
    res_next = sol.CreateVector()

    #create vectors for loop
    alpha=0
    u_next=sol.CreateVector()
    d_next=sol.CreateVector()
    beta=0
    p_next=sol.CreateVector()
    Cinvd=sol.CreateVector()
    Ap=sol.CreateVector()
    Cinvd_next=sol.CreateVector()
    iterator=0
    err=1000


    while err>tol and iterator<maxsteps:
        Cinvd.data=pre*d0.data
        Ap.data=mat*p0.data
        alpha=InnerProduct(d0.data,Cinvd.data)/InnerProduct(p0.data,Ap)
        u_next.data=u0.data+alpha*p0.data
        d_next.data=d0.data-alpha*Ap.data
        Cinvd_next.data=pre*d_next.data
        beta=InnerProduct(d_next.data, Cinvd_next.data)/InnerProduct(d0.data,Cinvd.data)
        p_next.data=Cinvd_next.data+beta*p0.data
        err=sqrt(InnerProduct(d_next.data,Cinvd_next.data))
        u0.data=u_next.data
        d0.data=d_next.data
        p0.data=p_next.data

        if(printrate):
            print("Iteration number:", iterator)
            print("Error after the iteration step:", err)
            #print(u_next.FV().NumPy()[:])

        iterator += 1

    return u_next

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

A = BilinearForm(fes)
F = LinearForm(fes)

A += grad(u)*grad(v)*dx

F += v*dx

A.Assemble()
F.Assemble()


c_jac = Preconditioner(A,"local")
c_jac.Update()

c_dir = Preconditioner(A, "direct")
c_dir.Update()

with TaskManager():
    gfu_new_1 = SolveCG(A.mat,c_jac.mat,gfu_jac.vec,F.vec,200,1e-16,True)
    
    gfu_new_2 = SolveCG(A.mat,c_dir.mat,gfu_dir.vec,F.vec,200,1e-16,True)

gif_jac = GridFunction(fes,name = "gif_jac")
gif_jac.vec.FV().NumPy()[:] = gfu_new_1

gif_dir = GridFunction(fes,name = "gif_dir")
gif_dir.vec.FV().NumPy()[:] = gfu_new_2
#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

Draw(gif_jac)
Draw(gif_dir)

In [ ]:
from ngsolve.solvers import CG

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

A = BilinearForm(fes)
F = LinearForm(fes)

A += grad(u)*grad(v)*dx

F += v*dx

A.Assemble()
F.Assemble()



c_dir = Preconditioner(A, "direct")
c_dir.Update()

data = []
callback = lambda k_jac,r_jac: data.append((k_jac,r_jac))

with TaskManager():
    #CG(A.mat,c_dir.mat,gfu_dir.vec,F.vec,200,1e-16,True)
    CG(mat=A.mat,pre=c_dir.mat,rhs=F.vec,sol=gfu_dir.vec,printrates=True,maxsteps=10000,tol=1e-16,callback = callback)
#Draw(gif_b)